In [13]:
import warnings
warnings.filterwarnings("ignore")



importing and installing libraries for exteacting the images

In [14]:
pip install PyMuPDF


DEPRECATION: Loading egg at c:\users\91945\appdata\local\programs\python\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..

[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
import fitz 

Extracting images from pdf using pyMuPDF and storing the all extracted images in a folder named extractedimage 

In [16]:
import os
import fitz 

def pdf_extracted_images(pdf_path, output_folder):
   
    os.makedirs(output_folder, exist_ok=True)
    doc = fitz.open(pdf_path)
    for i in range(len(doc)):
        page = doc.load_page(i)
        images = page.get_images(full=True)
        for img_index, img in enumerate(images):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image_ext = base_image["ext"]
            image_name = f"page_{i+1}_img_{img_index+1}.{image_ext}"
            with open(f"{output_folder}/{image_name}", "wb") as img_file:
                img_file.write(image_bytes)


pdf_extracted_images("well_student.pdf", "extractedimage")

In [17]:
pip install PyPDF2


Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\91945\appdata\local\programs\python\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..

[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
pip install Pillow


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: Pillow in c:\users\91945\appdata\local\programs\python\python311\lib\site-packages (10.0.0)



DEPRECATION: Loading egg at c:\users\91945\appdata\local\programs\python\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..

[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
import os
import fitz  
import PyPDF2
from PIL import Image

removing spilited images 

In [20]:
def remove_spilited_image(folder_path, min_width, min_height):
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            file_path = os.path.join(folder_path, filename)
            with Image.open(file_path) as img:
                width, height = img.size
            if width < min_width or height < min_height:
                os.remove(file_path)
                
output_folder = "extractedimage"
min_width = 55  
min_height = 55  

os.makedirs(output_folder, exist_ok=True)

remove_spilited_image(output_folder, min_width, min_height)

implementing different augmentation technique to remove currupted images.

In [21]:
import fitz 
import os
from PIL import Image
import random

def extract_augmentedimages_with_text(pdf_path, output_folder, margin=100, min_dimension=65):
    doc = fitz.open(pdf_path)
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        images = page.get_images(full=True)
        
        for img_index, img in enumerate(images):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image_ext = base_image["ext"]
            image_name = f"page_{page_num+1}_img_{img_index+1}.{image_ext}"
            
           
            img_rect = page.get_image_rects(xref)[0]
            
            
            width, height = img_rect.width, img_rect.height
            if width > min_dimension and height > min_dimension:
                
                original_image_path = os.path.join(output_folder, f"original_{image_name}")
                with open(original_image_path, "wb") as img_file:
                    img_file.write(image_bytes)
                
                
                text_rect = fitz.Rect(
                    img_rect.x0 - margin,
                    img_rect.y0 - margin,
                    img_rect.x1 + margin,
                    img_rect.y1 + margin
                )
                
                
                text_around_image = page.get_text("text", clip=text_rect)
                text_file_name = f"{output_folder}/original_{image_name.split('.')[0]}_text.txt"
                
                
                with open(text_file_name, "w", encoding="utf-8") as text_file:
                    text_file.write(text_around_image)
                
                
                augmented_images = augment_image(Image.open(original_image_path))
                
                
                for i, augmented_image in enumerate(augmented_images):
                    augmented_image_name = f"augmented_{i+1}_{image_name}"
                    augmented_image_path = os.path.join(output_folder, augmented_image_name)
                    augmented_image.save(augmented_image_path)
                    
                    augmented_text_file_name = f"{output_folder}/{augmented_image_name.split('.')[0]}_text.txt"
                    with open(augmented_text_file_name, "w", encoding="utf-8") as text_file:
                        text_file.write(text_around_image)
                    
            else:
                print(f"Ignored image {image_name} due to insufficient dimensions.")

def augment_image(image):
    augmented_images = []
    
    # Original image
    augmented_images.append(image.copy())
    
    # Rotation
    angle = random.uniform(-30, 30)
    augmented_images.append(image.rotate(angle))
    
    # Resizing
    width_factor = random.uniform(0.8, 1.2)
    height_factor = random.uniform(0.8, 1.2)
    new_width = int(image.width * width_factor)
    new_height = int(image.height * height_factor)
    if new_width > 0 and new_height > 0:
        resized_image = image.resize((new_width, new_height))
        augmented_images.append(resized_image)
    
    # Shearing
    shear_factor = random.uniform(-0.2, 0.2)
    width, height = image.size
    xshift = abs(shear_factor) * width
    new_width = width + int(round(xshift))
    if new_width > 0 and height > 0:
        shear_image = image.transform((new_width, height), Image.AFFINE, (1, shear_factor, -xshift if shear_factor > 0 else 0, 0, 1, 0), Image.BICUBIC)
        augmented_images.append(shear_image)
    
    return augmented_images


pdf_path = "well_student.pdf"
output_folder = "image&caption"
margin = 50  
min_dimension = 65  

os.makedirs(output_folder, exist_ok=True)

extract_augmentedimages_with_text(pdf_path, output_folder, margin, min_dimension)


Ignored image page_1_img_1.jpeg due to insufficient dimensions.


Renaming image with their corresponding captions

In [22]:
import os

def rename_images_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith('_text.txt'):
            text_file_path = os.path.join(folder_path, filename)
            
          
            image_extensions = ['.jpeg', '.jpg', '.png']
            img = None
            
            for ext in image_extensions:
                potential_image_path = text_file_path.replace('_text.txt', ext)
                if os.path.exists(potential_image_path):
                    img = potential_image_path
                    break
            
            if img and os.path.exists(text_file_path):
                try:
                    with open(text_file_path, 'r') as file:
                        text = file.read().strip()

                    if text:
                        new_name = text.splitlines()[0].strip()
                        valid_chars = "-_.() abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
                        sanitized_name = ''.join(c for c in new_name if c in valid_chars)
                        
                        new_image_file_path = os.path.join(os.path.dirname(img), sanitized_name + os.path.splitext(img)[1])
                        
                        os.rename(img, new_image_file_path)
                        print(f"Renamed image file to: {new_image_file_path}")
                    else:
                        print(f"No valid text found in {text_file_path}. Image file name remains unchanged.")
                except PermissionError as e:
                    print(f"Permission error: {e}")
                except Exception as e:
                    print(f"An error occurred: {e}")
            else:
                print(f"Text file or image file not found: {text_file_path}")

# Replace 'your_folder_path' with the actual path to your folder
folder_path = r'image&caption'
rename_images_in_folder(folder_path)


Renamed image file to: E:\DESKTOP\FOD\Internship Task\ongc internship\Image extraction and classification\image&caption\completions are used in areas where workover co.png
Renamed image file to: E:\DESKTOP\FOD\Internship Task\ongc internship\Image extraction and classification\image&caption\are usually best. In very deep wells single completion.png
Renamed image file to: E:\DESKTOP\FOD\Internship Task\ongc internship\Image extraction and classification\image&caption\economics of using multiple completions can be attractive. How.png
Renamed image file to: E:\DESKTOP\FOD\Internship Task\ongc internship\Image extraction and classification\image&caption\completion is used primarily in unconsolidated for.png
Renamed image file to: E:\DESKTOP\FOD\Internship Task\ongc internship\Image extraction and classification\image&caption\Figure 6 Showing a perforated cased hole completion for LPLT..jpeg
Renamed image file to: E:\DESKTOP\FOD\Internship Task\ongc internship\Image extraction and classific

using nlp and spacy library to classify image based on text(i.e caption)

In [23]:
import os
import shutil
import spacy


In [24]:
nlp = spacy.load("en_core_web_lg")

def categorize_images(image_folder):
    image_files = os.listdir(image_folder)
    categories = [
        "location map", "geological map", "structural map", "seismic section", 
        "log motive", "well construction diagram", "geotechnical order",
        "remote sensing image", "contour maps", "drilling plot", 
        "stratigraphy and casing plot"
    ]

    
    category_dir = "categories/"
    if os.path.exists(category_dir):
        shutil.rmtree(category_dir)
    os.makedirs(category_dir)

    for category in categories:
        os.makedirs(os.path.join(category_dir, category))

 
    os.makedirs(os.path.join(category_dir, "other"))

    for file in image_files:
        if file.endswith(('.jpeg', '.png')):  
            cleaned_name = file.lower().replace(".jpeg", "").replace(".png", "").replace("_", " ")
            doc_nlp = nlp(cleaned_name)
            
            if not doc_nlp.has_vector:
                print(f"Warning: The document '{cleaned_name}' has no vector representation.")
                shutil.copy(os.path.join(image_folder, file), os.path.join(category_dir, "other", file))
                continue

            best_category = None
            highest_sim = 0

            for category in categories:
                category_nlp = nlp(category)
                
                if not category_nlp.has_vector:
                    print(f"Warning: The category '{category}' has no vector representation.")
                    continue
                
                sim = doc_nlp.similarity(category_nlp)
                
                if sim > highest_sim:
                    highest_sim = sim
                    best_category = category

            if best_category and highest_sim > 0.5: 
                shutil.copy(os.path.join(image_folder, file), os.path.join(category_dir, best_category, file))
            else:
                shutil.copy(os.path.join(image_folder, file), os.path.join(category_dir, "other", file))


image_folder = "image&caption"
categorize_images(image_folder)